# Modelo NLP - Twitter

In [ ]:
# Installs
#!pip3 install scikit-learn
#!pip3 install -U spacy
#!python3 -m spacy download es
#!python3 -m spacy download es_core_news_sm

In [ ]:
#!pip install spacy

In [ ]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download es_core_news_sm

In [63]:
import pandas as pd
from spacy import displacy
import spacy
import re
import mysql.connector as conn

In [15]:
from spacy.lang.es.stop_words import STOP_WORDS

## Test Final

In [31]:
final_final_final_resultado = pd.read_csv('final_final_resultado.csv')

for line in final_final_final_resultado:
	#Reemplazar los caracteres unicode, no se dejaron tildes porque causan error
    strtmp1 = line.replace('\\u00f1', 'ñ')
    strtmp1 = strtmp1.replace('\\u00e1', 'a')
    strtmp1 = strtmp1.replace('\\u00e9', 'e')
    strtmp1 = strtmp1.replace('\\u00ed', 'i')
    strtmp1 = strtmp1.replace('\\u00f3', 'o')
    strtmp1 = strtmp1.replace('\\u00fa', 'u')
    strtmp1 = strtmp1.replace('\\u00bf', '¿')
    strtmp1 = strtmp1.replace('\\u00a1', '¡')
    strtmp1 = strtmp1.replace('\\u00d1', 'Ñ')
    strtmp1 = strtmp1.replace('\\u00c1', 'A')
    strtmp1 = strtmp1.replace('\\u00c9', 'E')
    strtmp1 = strtmp1.replace('\\u00cd', 'I')
    strtmp1 = strtmp1.replace('\\u00d3', 'O')
    strtmp1 = strtmp1.replace('\\u00da', 'U')
    strtmp1 = strtmp1.replace('\\u00fc', 'ü')
    strtmp1 = strtmp1.replace('\\u00b0', '')
    #Quitar el inicio y el fin del json para dejar solo los tweets
    strtmp1 = strtmp1.replace('{"tweets": {"tweet": ', '')
    strtmp1 = strtmp1.replace(']}}', ']')
    #Quitar el diccionario que contiene la polaridad y dejarla solo con su valor de sentimiento
    strtmp1 = strtmp1.replace('"sentiment": {"polarity": {"value": ', '"sentiment": ')
    strtmp1 = strtmp1.replace('"NONE"}}', '"NONE"')
    #Asignamos al sentimiento positivo el valor de 1
    strtmp1 = strtmp1.replace('"P"}}', '1')
    strtmp1 = strtmp1.replace('"NEU"}}', '"NEU"')
    #Asignamos al sentimiento negativo el valor de 0
    strtmp1 = strtmp1.replace('"N"}}', '0')
    #eliminación de puntuaciones
    strtmp1 = re.sub('[¡!#$).;¿?&°]', '', strtmp1.lower())


In [32]:
#Quitar columnas sin clasificación de sentimiento 
indexNames = final_final_final_resultado[(final_final_final_resultado['sentiment'] == 'none') | (final_final_final_resultado['sentiment'] == 'neu')].index
final_final_final_resultado.drop(indexNames , inplace=True)
final_final_final_resultado.head()

,tweetid,user,content,date,lang,sentiment
0,768213567418036224,3429794128,@myendlesshazza a que puto mal escribo b me si...,2016-08-23 22:29:21+00:00,es,0
1,768221670255493120,396616007,quiero mogollon a @albabenito99 pero sobretodo...,2016-08-23 23:01:33+00:00,es,1
2,768221021300264960,2845050061,vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58+00:00,es,0
3,768220253730009088,442100979,@yulian_poe @guillermoterry1 ah mucho mas por ...,2016-08-23 22:55:55+00:00,es,1
4,768231229439311872,529648312,@toni_end seria mejor que dejasen de emitir es...,2016-08-23 23:39:32+00:00,es,0


In [33]:
#Importar librerías de aprendizaje
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [34]:
#Verificar frecuencias de cada categoría
final_final_final_resultado['sentiment'].value_counts()

0    2854
1    1896
Name: sentiment, dtype: int64

In [35]:
#Verificar si hay datos nulos
final_final_final_resultado.isnull().sum()

tweetid      0
user         0
content      0
date         0
lang         0
sentiment    0
dtype: int64

# Tokenización

In [36]:
import string #Quita todos los simbolos
puntua = string.punctuation + "¡¿"
puntua

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡¿'

In [37]:
#Función para limpieza de datos
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.strip()
        else:
            temp = token
        tokens.append(temp)
    
    clean_tokens = []
    for token in tokens:
        if token not in stopwords_spacy and token not in puntua:
            clean_tokens.append(token)
    
    return clean_tokens

In [38]:
text_data_cleaning("¡Hola cómo estás!. ¿Te gusta el meetup?")

['Hola', 'Te', 'gustar', 'meetup']

# Vectorization Feature Engineering (TF-IDF)

In [39]:
#importar librería de vectorización
from sklearn.svm import LinearSVC

In [40]:
#Definir la función de tokenizado y crear el clasificador lineal
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [41]:
#Crear los vectores de datos
X = final_final_final_resultado['content']
y = final_final_final_resultado['sentiment']

In [42]:
#Crear el vector de entrenamiento como una porción de los datos y dejar el resto para pruebas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3800,), (950,), (3800,), (950,))

In [43]:
#Crear un pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [44]:
#evitar que el formato se tome como unknown
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [45]:
#Entrenar el clasificador
clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7fedfeb39670>)),
                ('clf', LinearSVC())])

In [46]:
#Crear el vectos de valores predichos a partir del clasificador
y_pred = clf.predict(X_test)

In [47]:
#Ver la precisión obtenida
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88       578
           1       0.84      0.78      0.81       372

    accuracy                           0.86       950
   macro avg       0.85      0.84      0.85       950
weighted avg       0.86      0.86      0.85       950



In [48]:
# 86% de 'efectividad'

In [49]:
#Crear la matriz de confusión
confusion_matrix(y_test, y_pred)

array([[522,  56],
       [ 81, 291]])

Me dan 82 falsos negativos

In [ ]:
#Predecir algunas frases de prueba
clf.predict(['Eres una calamidad'])

## Creo base de datos necesarias

### Creando SQL Database

In [67]:
db=conn.connect(host='localhost', user='root', passwd='123456', database='hate_blocker')

cursor=db.cursor()

cursor.execute('drop table if exists hate_blocker')

### Creando Tabla: Train Model

El objetivo de este tabla es guardar los siguientes tweets:

- Tweets correctos identificados como sentiment 1.

- Tweets incorrectos identificados como sentiment 0.

- Tweets correctos identificados como sentiment 0.

In [68]:
tabla= '''
        create table train_model(
            
           tweet_id bigint ,      
           user bigint,
           content varchar(500),
           date varchar(500),
           lang varchar(500),
           sentiment bigint
            
        );
'''

cursor.execute (tabla)

### Creando Tabla: Listado Bullies

In [ ]:
tabla= '''
        create table listado_bullies(
            
           
           tweet_id bigint,
           date varchar(500),      
           name varchar(500),
           account varchar(500),
           tweet_text varchar(500),
           URL_account varchar(500),
           URL_tweet varchar(500)
            
        );
'''


cursor.execute (tabla)

# Función-Modelo

In [51]:
import tweepy

In [59]:
# 4 authentication chains

consumer_key='Qbxaw0OrPeZSFljKEp6SHI19M'
consumer_secret='kva4UfB6cK2Z6P70MGV8LAM0MMOu3SAHjuUl5t7bWxpegkCHGl'
access_key='902474996-eLIpuPP1uqYZoTWtprZuiL2FmehUURAZHT8ZBReU'
access_secret='CkVPZlMz5Lk4EMYDZzmNM2zJQHGE0ze6CA0auHqnYtEhU'

# authorize twitter, initialize tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [98]:

def insert(sentiment):
    
    insert_query="insert into {} ({}) values {};"\
                        .format('train_model', ','.join(['tweet_id', 'user', 'content', 'date', 'lang', 'sentiment']), tuple([tweet.id_str, i.id,tweet.text, str(tweet.created_at), 'es', sentiment]))


    cursor.execute(insert_query)

    db.commit()

    return

In [102]:
def bullies_to_sql():
    
    insert_query="insert into {} ({}) values {};"\
                        .format('listado_bullies', ','.join(['tweet_id','date', 'name', 'account', 'tweet_text', 'URL_account','URL_tweet']), 
                                tuple([tweet.id_str, str(tweet.created_at), i.name, i.screen_name,tweet.text, 'https://twitter.com/' + i.screen_name, 'https://twitter.com/' + i.screen_name + '/status/' + tweet.id_str]))


    cursor.execute(insert_query)

    db.commit()

    return

In [97]:
users = []
users_url = []
tweets_url = []

mentions = api.mentions_timeline(tweet_mode= 'extented')

for tweet in mentions: # Entran los tweets
    
    print(tweet)
    users.append(tweet.user)

    for i in users:
        
        print(i)

    
    
    valoracion = clf.predict([tweet.text]) 
    
    if valoracion == 1: # Es positivo el tweet?
        
        print ('Es buena gente')
        
        insert(1) # Mete en sql los tweets con etiqueta 1 para entrenar.

        
    elif valoracion == 0: # Es negativo el tweet?
        
        print('Voy a bloquear a ', tweet.user.screen_name, 'por el siguiente tweet:', tweet.text )
               
        verif = input ('¿Deseas bloquear a este usuario?') # Estas seguro que quieres bloquear?
        
        print (verif)
        
        if verif == 'Si': # Si
            
            api.create_block(tweet.user.screen_name)
            
            insert(0) # Mete en sql los tweets con etiqueta 0 para entrenar.
            
            print('Bloqueado.')

            
        elif verif == 'No': # No
            
            insert(1) # Mete en sql los tweets con etiqueta 1 para entrenar.
            
            print ('Perdón por las molestias. Tengo mucho que aprender.')

Status(_api=<tweepy.api.API object at 0x7fedff351580>, _json={'created_at': 'Sat Jun 19 19:39:18 +0000 2021', 'id': 1406335802897014796, 'id_str': '1406335802897014796', 'text': '@xinxinricardo Vamooooo', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'xinxinricardo', 'name': 'Esqueje (lo de la rama)', 'id': 902474996, 'id_str': '902474996', 'indices': [0, 14]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': 1406333542448218115, 'in_reply_to_status_id_str': '1406333542448218115', 'in_reply_to_user_id': 902474996, 'in_reply_to_user_id_str': '902474996', 'in_reply_to_screen_name': 'xinxinricardo', 'user': {'id': 54359433, 'id_str': '54359433', 'name': 'Javi Rando', 'screen_name': 'javirando', 'location': 'Málaga', 'description': 'Creador y todo lo que se te ocurra en @JornadaPerfecta. CEO en Fantasy Content. Papá de Javi y Olivia', 'url': 'https://t.c

KeyboardInterrupt: Interrupted by user

In [85]:
type(tweet)

tweepy.models.Status

In [ ]:
insert_query="insert into {} ({}) values {};"\
                    .format('pabloiglesias_tweets', ','.join(['tweet_id','date',
           'tweet_text','source','in_reply_to_screen_name','user_name',
           'user_screen_name','location','user_url','n_followers','verified']), tuple([tweet_id,date,tweet_text,
                                                                                      source, in_reply_to_screen_name,name,
                                                                                      screen_name,location,url,followers_count,verified]))
        #print (insert_query)
        
        cursor.execute(insert_query)
    
        db.commit()